In [1]:
import psycopg2 as pg
import pandas as pd
from queries.db_queries import *
from functions.running_avg import *
import numpy as np
#import libraries.libraries

In [2]:
# Retrosheet contains data from teams who have moved/renamed. Therefore, some abbreviations differ from Lahman DB. This dict maps them
renamed_team = {
    'NYN': 'NYM',
    'CHA': 'CHI',
    'SFN': 'SFG',
    'ANA': 'LAA',
    'CHN': 'CHC',
    'TBA': 'TBR',
    'LAN': 'LAD',
    'SDN': 'SDP',
    'WAS': 'WSN',
    'SLN': 'STL',
    'KCA': 'KCR',
    'NYA': 'NYY',
}

In [3]:
game_log_column_headers = [
    "Date",
    "Game Number",
    "Day of Week",
    "Visiting Team",
    "Visiting Team League",
    "Visiting Team Game Number",
    "Home Team",
    "Home Team League",
    "Home Team Game Number",
    "Visiting Team Score",
    "Home Team Score",
    "Length of Game (Outs)",
    "Day/Night Indicator",
    "Park ID",
    "Attendance",
    "Time of Game"
]

In [4]:
# Step 1 : Call Funciton to Return all Parks
# Step 2 : Loop through each park calculating metrics for each
# Step 3 : Devise way to normalize
# Step 4 : Compare to Park Factors from Lahman
# Step 5 :
# Step 6 : Use New metric to predict player performance
team_data = sql_query(curr_teams_query)

## Get Game Logs and Rename Abbr.

In [4]:
log_path_2023 = 'C:/Users/conor/Downloads/gl2023/gl2023.txt'

gamelogs = pd.read_csv(log_path_2023, header=None)
gamelogs_ = pd.concat([gamelogs.iloc[:,:13], gamelogs.iloc[:,16:19]], axis = 1)
gamelogs_ = gamelogs_.set_axis(game_log_column_headers, axis = 1)

#Fix Abbreviations
gamelogs_['Visiting Team'] = gamelogs_['Visiting Team'].replace(renamed_team)
gamelogs_['Home Team'] = gamelogs_['Home Team'].replace(renamed_team)

In [ ]:
Phils_Games = gamelogs_[(gamelogs_['Home Team'] == 'PHI') |(gamelogs_['Visiting Team'] == 'PHI') ]
Opp_Score = []

running_avg_df = get_running_avg_df(gamelogs_=gamelogs_, timeframe=15)

for idx, game in Phils_Games.iterrows():
    
    if game['Home Team'] == 'PHI':
        opponent = game['Visiting Team']
        score = 'VTRA_Norm'
        homeaway = 'Visiting Team'
    else:
        opponent = game['Home Team']
        score = 'HTRA_Norm'
        homeaway = 'Home Team'    
    date = game.Date
    opp_score = running_avg_df[(running_avg_df['Date']== date) & (running_avg_df[homeaway]== opponent)][score].values
    Opp_Score.append(np.mean(opp_score) if len(opp_score) > 0 else None)
Phils_Games['Game Difficulty'] = Opp_Score


In [6]:
Phils_Games.reset_index(drop=True, inplace=True)
Phils_Games.head()

,Date,Game Number,Day of Week,Visiting Team,Visiting Team League,Visiting Team Game Number,Home Team,Home Team League,Home Team Game Number,Visiting Team Score,Home Team Score,Length of Game (Outs),Day/Night Indicator,Park ID,Attendance,Time of Game,Game Difficulty
0,20230330,0,Thu,PHI,NL,1,TEX,AL,1,7,11,51,D,ARL03,38387.0,184,0.814815
1,20230401,0,Sat,PHI,NL,2,TEX,AL,2,3,16,51,D,ARL03,31916.0,190,1.000000
2,20230402,0,Sun,PHI,NL,3,TEX,AL,3,1,2,51,N,ARL03,25823.0,144,0.716049
3,20230403,0,Mon,PHI,NL,4,NYY,AL,4,1,8,51,N,NYC21,37202.0,161,0.444444
4,20230404,0,Tue,PHI,NL,5,NYY,AL,5,4,1,54,N,NYC21,35392.0,165,0.370370
